### Import

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

from tqdm import tqdm
import numpy as np
import math
import matplotlib.pyplot as plt
from itertools import combinations_with_replacement
from numpy.linalg import qr, norm
import open3d as o3d
import datetime
from utils import Logger, mkdir_p, progress_bar, save_model, save_args, cal_loss

import sklearn.metrics as metrics
from data import ModelNet40
from torch.optim.lr_scheduler import CosineAnnealingLR
import sklearn.metrics as metrics
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from itertools import combinations_with_replacement

def generate_monomials_sequences_batch(X, degree):
    """
    Generate monomials up to a given degree for d-dimensional points.

    Parameters:
        X (ndarray): An array of shape (B, N, d) representing the point cloud.
        degree (int): The maximum degree of the monomials.

    Returns:
        ndarray: Monomial matrix of shape (B, N, M).
    """
    B, N, d = X.shape
    device = X.device
    exponents = []
    # Generate all multi-indices with total degree up to 'degree'
    for total_degree in range(degree + 1):
        for exps in combinations_with_replacement(range(d), total_degree):
            exp = torch.zeros(d, dtype=int, device=device)
            for var in exps:
                exp[var] += 1
            exponents.append(exp)
    M = len(exponents)
    # print(f'Number of monomials: {M}') # Number of polynomials: n1 + n2 + ... + n_d = degree; degree + d choose d; d number of dividers for an array in space R^d.
    monomials = torch.ones(B, N, M, device=device)
    for i, exp in enumerate(exponents):
        monomials[:, :, i] = torch.prod(X ** exp, axis=2) # x1^exp1 * x2^exp2 * ... * xd^expd. e.g. x1^2 * x2^3 * x3^1 \in R^3
    return monomials.sum(dim=1) / N # (B, N, M) -> (B, M)

def generate_chebyshev_polynomials_sequence_batch(X, degree):
    """
    Generate Chebyshev polynomials up to a given degree for d-dimensional points.

    Parameters:
        X (ndarray): An array of shape (B, N, d) representing the d-dimensional point cloud.
        degree (int): The maximum degree of the polynomials.

    Returns:
        ndarray: Chebyshev polynomial matrix of shape (B, N, M).
    """
    B, N, d = X.shape
    device = X.device
    exponents = []
    # Generate all multi-indices with total degree up to 'degree'
    for total_degree in range(degree + 1):
        for exps in combinations_with_replacement(range(d), total_degree):
            exp = torch.zeros(d, dtype=int, device=device)
            for var in exps:
                exp[var] += 1
            exponents.append(exp)
    M = len(exponents)
    exponent = torch.tensor(exponents, device=device) # (M, d)
    # print(f'Number of Chebyshev polynomials: {M}') # Number of polynomials: n1 + n2 + ... + n_d = degree; degree + d choose d; d number of dividers for an array in space R^d.
    # cheby_polynomials = torch.ones(B, N, M, device=device)
    # for i, exp in enumerate(exponents):
    #     cheby_polynomials[:, :, i] = torch.prod(torch.cos(exp * torch.acos(X)), axis=2)
    cheby_polynomials = torch.cos(exponent.unsqueeze(0) * torch.acos(X)) # (B, N, M)
    cheby_polynomials = cheby_polynomials.prod(dim=2) # (B, N)
    
    
    return cheby_polynomials.sum(dim=1) / N # (B, N, M) -> (B, M)

def poly_seq_batch(X, degree, poly_type='monomials'):
    if poly_type == 'monomials':
        return generate_monomials_sequences_batch(X, degree)
    elif poly_type == 'chebyshev':
        return generate_chebyshev_polynomials_sequence_batch(X, degree)
    else:
        raise ValueError('Invalid polynomial type')

### Use ModelNet40 as our dataset

In [3]:
num_points = 2048
batch_size = 32

train_loader = DataLoader(ModelNet40(partition='train', 
                                        num_points=num_points,
                                        aug=True), 
                            num_workers=8,
                            batch_size=batch_size, 
                            shuffle=True, 
                            drop_last=True)
test_loader = DataLoader(ModelNet40(partition='test', 
                                        num_points=num_points,
                                        aug=False), 
                            num_workers=8,
                            batch_size=batch_size // 2, 
                            shuffle=False, 
                            drop_last=False)

### A simple Linear classifier

In [4]:
class MLPClaasifier(nn.Module):
    def __init__(self, dim_in, layer_dims, dim_out, dropout=0.5):
        super(MLPClaasifier, self).__init__()
        self.convs = nn.ModuleList()
        self.convs.append(nn.Sequential(
            nn.Linear(dim_in, layer_dims[0]),
            nn.BatchNorm1d(layer_dims[0]),
            nn.ReLU(),
        ))
        for i in range(1, len(layer_dims)):
            self.convs.append(nn.Sequential(
                nn.Linear(layer_dims[i-1], layer_dims[i]),
                nn.BatchNorm1d(layer_dims[i]),
                nn.Dropout(dropout),
                nn.ReLU(),
            ))
        self.convs.append(nn.Linear(layer_dims[-1], dim_out))

    def forward(self, x):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x)
        x = self.convs[-1](x)
        return x
    
x = torch.randn(32, 100) # 32 samples, 100 features
model = MLPClaasifier(100, [128, 64], 40)
y = model(x)
print(y.shape)

torch.Size([32, 40])


In [5]:
class MomentNet(nn.Module):
    def __init__(self, dim_in, layer_dims, dim_out, degree, poly_type='monomials'):
        super(MomentNet, self).__init__()
        self.poly_type = poly_type
        self.degree = degree
        self.num_poly = math.comb(degree + dim_in, dim_in)
        self.mlp = MLPClaasifier(self.num_poly, layer_dims, dim_out)
        
    def forward(self, x):
        x = poly_seq_batch(x, self.degree, self.poly_type)
        x = self.mlp(x)
        return x
    
x = torch.randn(32, 2048, 3) # 32 samples, 2048 points, 3 features
model = MomentNet(3, [128, 64], 40, 2, 'monomials')
y = model(x)
print(y.shape)

torch.Size([32, 40])


In [6]:
def train(net, trainloader, optimizer, criterion, epoch, num_epochs, device):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    train_pred = []
    train_true = []
    time_cost = datetime.datetime.now()
    pbar = tqdm(enumerate(trainloader), total=len(trainloader))
    for batch_idx, (data, label) in pbar:
        data, label = data.to(device), label.to(device).squeeze()
        optimizer.zero_grad()
        logits = net(data)
        loss = criterion(logits, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(), 1)
        optimizer.step()
        train_loss += loss.item()
        preds = logits.max(dim=1)[1]

        train_true.append(label.cpu().numpy())
        train_pred.append(preds.detach().cpu().numpy())

        total += label.size(0)
        correct += preds.eq(label).sum().item()

        pbar.set_description(f'({epoch:3d}/{num_epochs:3d}) Loss: {train_loss / (batch_idx + 1):.3f} | Acc: {100. * correct / total:.3f}% ({correct}/{total})')
        # progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
        #              % (train_loss / (batch_idx + 1), 100. * correct / total, correct, total))

    time_cost = int((datetime.datetime.now() - time_cost).total_seconds())
    train_true = np.concatenate(train_true)
    train_pred = np.concatenate(train_pred)
    return {
        "loss": float("%.3f" % (train_loss / (batch_idx + 1))),
        "acc": float("%.3f" % (100. * metrics.accuracy_score(train_true, train_pred))),
        "acc_avg": float("%.3f" % (100. * metrics.balanced_accuracy_score(train_true, train_pred))),
        "time": time_cost
    }


def validate(net, testloader, criterion, epoch, num_epochs, device):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    test_true = []
    test_pred = []
    time_cost = datetime.datetime.now()
    with torch.no_grad():
        pbar = tqdm(enumerate(testloader), total=len(testloader))
        for batch_idx, (data, label) in pbar:
            data, label = data.to(device), label.to(device).squeeze()
            logits = net(data)
            loss = criterion(logits, label)
            test_loss += loss.item()
            preds = logits.max(dim=1)[1]
            test_true.append(label.cpu().numpy())
            test_pred.append(preds.detach().cpu().numpy())
            total += label.size(0)
            correct += preds.eq(label).sum().item()
            pbar.set_description(f'({epoch:3d}/{num_epochs:3d}) Loss: {test_loss / (batch_idx + 1):.3f} | Acc: {100. * correct / total:.3f}% ({correct}/{total})')
            # progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            #              % (test_loss / (batch_idx + 1), 100. * correct / total, correct, total))

    time_cost = int((datetime.datetime.now() - time_cost).total_seconds())
    test_true = np.concatenate(test_true)
    test_pred = np.concatenate(test_pred)
    return {
        "loss": float("%.3f" % (test_loss / (batch_idx + 1))),
        "acc": float("%.3f" % (100. * metrics.accuracy_score(test_true, test_pred))),
        "acc_avg": float("%.3f" % (100. * metrics.balanced_accuracy_score(test_true, test_pred))),
        "time": time_cost
    }

In [7]:
# Training (Taken from PointMLP-PyTorch)
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
num_epochs = 100 # Number of epochs
eval_every = 10 # Evaluate every 'eval_every' epochs
max_lr = 0.1 # Maximum learning rate
min_lr = 0.005 # Minimum learning rate

best_test_acc = 0.  # best test accuracy
best_train_acc = 0.
best_test_acc_avg = 0.
best_train_acc_avg = 0.
best_test_loss = float("inf")
best_train_loss = float("inf")
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Model
net = MomentNet(3, [256, 128], 40, degree = 14, poly_type = 'chebyshev').to(device)
print(net.num_poly)

# Optimizer
optimizer = torch.optim.SGD(net.parameters(), lr=max_lr, momentum=0.9, weight_decay=0)
scheduler = CosineAnnealingLR(optimizer, num_epochs, eta_min=min_lr, last_epoch=start_epoch - 1)
criterion = nn.CrossEntropyLoss()

for epoch in range(start_epoch, num_epochs):
    train_out = train(net, train_loader, optimizer, criterion, epoch, num_epochs, device)  # {"loss", "acc", "acc_avg", "time"}
    test_out = validate(net, test_loader, criterion, epoch, num_epochs, device)
    scheduler.step()
    if test_out["acc"] > best_test_acc:
        best_test_acc = test_out["acc"]
        is_best = True
    else:
        is_best = False

    best_test_acc = test_out["acc"] if (test_out["acc"] > best_test_acc) else best_test_acc
    best_train_acc = train_out["acc"] if (train_out["acc"] > best_train_acc) else best_train_acc
    best_test_acc_avg = test_out["acc_avg"] if (test_out["acc_avg"] > best_test_acc_avg) else best_test_acc_avg
    best_train_acc_avg = train_out["acc_avg"] if (train_out["acc_avg"] > best_train_acc_avg) else best_train_acc_avg
    best_test_loss = test_out["loss"] if (test_out["loss"] < best_test_loss) else best_test_loss
    best_train_loss = train_out["loss"] if (train_out["loss"] < best_train_loss) else best_train_loss
    
    print(
        f"Training loss:{train_out['loss']} acc_avg:{train_out['acc_avg']}% acc:{train_out['acc']}% time:{train_out['time']}s")
    print(
        f"Testing loss:{test_out['loss']} acc_avg:{test_out['acc_avg']}% "
        f"acc:{test_out['acc']}% time:{test_out['time']}s [best test acc: {best_test_acc}%]")
    

680


(  0/100) Loss: 1.638 | Acc: 56.158% (5517/9824): 100%|██████████| 307/307 [00:23<00:00, 13.05it/s]
(  0/100) Loss: 0.977 | Acc: 72.366% (1786/2468): 100%|██████████| 155/155 [00:11<00:00, 13.72it/s]

Training loss:1.638 acc_avg:39.928% acc:56.158% time:23s
Testing loss:0.977 acc_avg:62.128% acc:72.366% time:11s [best test acc: 72.366%]



(  1/100) Loss: 1.139 | Acc: 67.590% (6640/9824): 100%|██████████| 307/307 [00:22<00:00, 13.55it/s]
(  1/100) Loss: 0.871 | Acc: 74.757% (1845/2468): 100%|██████████| 155/155 [00:11<00:00, 13.96it/s]

Training loss:1.139 acc_avg:54.65% acc:67.59% time:23s
Testing loss:0.871 acc_avg:64.159% acc:74.757% time:11s [best test acc: 74.757%]


...

Training loss:0.188 acc_avg:90.204% acc:93.414% time:23s
Testing loss:0.748 acc_avg:80.978% acc:86.102% time:11s [best test acc: 86.305%]



( 98/100) Loss: 0.188 | Acc: 93.170% (9153/9824): 100%|██████████| 307/307 [00:23<00:00, 13.05it/s]
( 98/100) Loss: 0.742 | Acc: 85.737% (2116/2468): 100%|██████████| 155/155 [00:11<00:00, 13.45it/s]

Training loss:0.188 acc_avg:90.099% acc:93.17% time:23s
Testing loss:0.742 acc_avg:80.628% acc:85.737% time:11s [best test acc: 86.305%]



( 99/100) Loss: 0.187 | Acc: 93.149% (9151/9824): 100%|██████████| 307/307 [00:23<00:00, 12.95it/s]
( 99/100) Loss: 0.755 | Acc: 85.940% (2121/2468): 100%|██████████| 155/155 [00:11<00:00, 13.72it/s]

Training loss:0.187 acc_avg:90.147% acc:93.149% time:24s
Testing loss:0.755 acc_avg:81.078% acc:85.94% time:11s [best test acc: 86.305%]
